In [3]:
import torch
import sys
sys.path.append('e:/tennis_v2/')
import cv2
from ultralytics import RTDETR, SAM, FastSAM, YOLO
from utils.kalman_filter import KalmanFilter
import numpy as np
from __future__ import absolute_import
import numpy as np

%load_ext autoreload
%autoreload 2


In [59]:
reals = [[783.4759, 346.4225, 792.5747, 364.8918],
        [792.0668, 376.4739, 801.0619, 399.4743],
        [806.9440, 446.4276, 816.0681, 461.5112],
        [806.7212, 446.2137, 816.0037, 461.7293],
        [814.6136, 458.4625, 822.3652, 467.1291],
        [819.8138, 466.5891, 827.4417, 475.8541]]
kf = KalmanFilter()
kf._std_weight_position = 1. / 20
kf._std_weight_velocity = 1. / 290

for i in range(len(reals) - 1):
    if i == 0:
        kf_mean, kf_covariance = kf.initiate(kf.xyxy_to_xyah(reals[i]))
    kf_mean, kf_covariance = kf.predict(kf_mean, kf_covariance)
    real = reals[i + 1]
    iou = kf._compute_iou(kf.xyah_to_xyxy(kf_mean[:4]), real)
    print(f'第{i + 1} 帧预测, 与真实的值iou: {iou}')
    # 使用真实值校正
    kf_mean, kf_covariance = kf.update(kf_mean, kf_covariance, kf.xyxy_to_xyah(real))


第1 帧预测, 与真实的值iou: 0.0
第2 帧预测, 与真实的值iou: 0.0
第3 帧预测, 与真实的值iou: 0.07670186757253894
第4 帧预测, 与真实的值iou: 0.15222340932302345
第5 帧预测, 与真实的值iou: 0.1690883188380371


In [ ]:

kf._std_weight_position

0.5

In [62]:
mean, covariance = kf.initiate(kf.xyxy_to_xyah(real[0]))
mean, covariance

(array([     788.03,      355.66,     0.49264,      18.469,           0,           0,           0,           0]),
 array([[     13.645,           0,           0,           0,           0,           0,           0,           0],
        [          0,      13.645,           0,           0,           0,           0,           0,           0],
        [          0,           0,      0.0001,           0,           0,           0,           0,           0],
        [          0,           0,           0,      13.645,           0,           0,           0,           0],
        [          0,           0,           0,           0,       21831,           0,           0,           0],
        [          0,           0,           0,           0,           0,       21831,           0,           0],
        [          0,           0,           0,           0,           0,           0,       1e-10,           0],
        [          0,           0,           0,           0,           0,           0,  

In [63]:
mean, covariance = kf.update(mean, covariance, kf.xyxy_to_xyah(real[1]))
mean, covariance

(array([     794.86,      381.51,     0.49164,      22.094,           0,           0,           0,           0]),
 array([[     2.7289,           0,           0,           0,           0,           0,           0,           0],
        [          0,      2.7289,           0,           0,           0,           0,           0,           0],
        [          0,           0,   9.901e-05,           0,           0,           0,           0,           0],
        [          0,           0,           0,      2.7289,           0,           0,           0,           0],
        [          0,           0,           0,           0,       21831,           0,           0,           0],
        [          0,           0,           0,           0,           0,       21831,           0,           0],
        [          0,           0,           0,           0,           0,           0,       1e-10,           0],
        [          0,           0,           0,           0,           0,           0,  

In [64]:
# 预测的是第3帧
kf_mean, kf_covariance = kf.predict(mean, covariance)
kf_mean, kf_covariance

(array([     794.86,      381.51,     0.49164,      22.094,           0,           0,           0,           0]),
 array([[      21839,           0,           0,           0,       21831,           0,           0,           0],
        [          0,       21839,           0,           0,           0,       21831,           0,           0],
        [          0,           0,  0.00019901,           0,           0,           0,       1e-10,           0],
        [          0,           0,           0,       21839,           0,           0,           0,       21831],
        [      21831,           0,           0,           0,       22144,           0,           0,           0],
        [          0,       21831,           0,           0,           0,       22144,           0,           0],
        [          0,           0,       1e-10,           0,           0,           0,       2e-10,           0],
        [          0,           0,           0,       21831,           0,           0,  

In [65]:
kf.xyah_to_xyxy(kf_mean[:4])

[789.4253596569745, 370.46362, 800.2877203430255, 392.55780000000004]

In [66]:
real[2]

[806.944, 446.4276, 816.0681, 461.5112]

In [67]:
kf.xyah_to_xyxy(kf_mean[:4])

[789.4253596569745, 370.46362, 800.2877203430255, 392.55780000000004]

In [68]:
kf._compute_iou(kf.xyah_to_xyxy(kf_mean[:4]), real[2])

0.0

In [69]:
kf_mean, kf_covariance = kf.update(kf_mean, kf_covariance, real[2])
kf_mean, kf_covariance

(array([     806.94,      446.41,      16.406,      461.41,      12.081,       64.88,  7.9966e-06,      439.17]),
 array([[     4.8804,           0,           0,           0,      4.8787,           0,           0,           0],
        [          0,      4.8804,           0,           0,           0,      4.8787,           0,           0],
        [          0,           0,  0.00019513,           0,           0,           0,  9.8049e-11,           0],
        [          0,           0,           0,      4.8804,           0,           0,           0,      4.8787],
        [     4.8787,           0,           0,           0,       324.9,           0,           0,           0],
        [          0,      4.8787,           0,           0,           0,       324.9,           0,           0],
        [          0,           0,  9.8049e-11,           0,           0,           0,       2e-10,           0],
        [          0,           0,           0,      4.8787,           0,           0,  

In [70]:
# 预测第4帧
kf_mean, kf_covariance = kf.predict(kf_mean, kf_covariance)

In [71]:
kf_mean, kf_covariance

(array([     819.02,      511.29,      16.406,      900.58,      12.081,       64.88,  7.9966e-06,      439.17]),
 array([[     2468.6,           0,           0,           0,      329.78,           0,           0,           0],
        [          0,      2468.6,           0,           0,           0,      329.78,           0,           0],
        [          0,           0,  0.00029513,           0,           0,           0,  2.9805e-10,           0],
        [          0,           0,           0,      2468.6,           0,           0,           0,      329.78],
        [     329.78,           0,           0,           0,  1.3658e+05,           0,           0,           0],
        [          0,      329.78,           0,           0,           0,  1.3658e+05,           0,           0],
        [          0,           0,  2.9805e-10,           0,           0,           0,       3e-10,           0],
        [          0,           0,           0,      329.78,           0,           0,  

In [44]:
kf = KalmanFilter()
kf.xyxy_to_xyah([220, 220, 230, 230])

[225.0, 225.0, 1.0, 10]

In [39]:
video_input_path = 'e:/TennisProject-main/inference/test_02.mp4'
video_output_path = 'e:/tennis_v2/inference/output_3_' + video_input_path.split('/')[-1]
video_output_path

'e:/tennis_v2/inference/output_3_test_02.mp4'

In [20]:
def compute_mahalanobis(kf_mean, kf_covariance, detection_bbox):
    """
    计算检测框与KF预测状态之间的马氏距离
    """

    # 计算残差
    residual = detection_bbox - kf_mean[:4]
    # 计算马氏距离
    inv_cov = np.linalg.inv(kf_covariance[:4, :4])
    mahalanobis_dist = np.sqrt(residual.T @ inv_cov @ residual)
    return mahalanobis_dist

In [37]:
def process_video(video_input_path, video_output_path, model):
    kf = KalmanFilter()
    
    print(video_input_path)
    # 打开视频文件
    cap = cv2.VideoCapture(video_input_path)
    if not cap.isOpened():
        raise IOError("Cannot open the video file")

    # 获取视频参数以复制编解码器和帧率
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # 获取视频帧率

    # 初始化视频写入器
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_output_path, fourcc, 1, (frame_width, frame_height))

    frame_id = 0  # 当前帧 ID
    stable_frames = 0
    stable_frames_threshold = 1
    stable_ious_threshold = 0.2
    score_weight = 0.45

    kf_mean, kf_covariance = None, None
    while True:
        # 读取视频帧
        ret, frame = cap.read()
        if not ret:
            break  # 视频结束
        
        except_boxes = []
        last_frame_boxes = []

        results = model(frame)
        for result in results:

            boxes = result.boxes.xyxy.cpu().numpy()  # 所有边界框 [N,4]
            confidences = result.boxes.conf.cpu().numpy()  # 置信度 [N]
            class_ids = result.boxes.cls.cpu().numpy()  # 类别ID [N]

            # 过滤出目标类别（如class_id=32为球）
            ball_mask = (class_ids == 32)
            ball_boxes = boxes[ball_mask]
            ball_confidences = confidences[ball_mask]
            
            if 32 in result.boxes.cls:
                for _, (bbox, conf) in enumerate(zip( ball_boxes, ball_confidences)):
                    
                    # 与上一帧进行比较，iou值很大的物体，也就是不动的物体也不是需要追踪的网球
                    if len(last_frame_boxes) > 0:
                        last_frame_iou = max(kf.compute_iou(bbox, last_frame_boxes))
                        if last_frame_iou > 0.5:
                            # 检测出的和之前的box基本重复，也就是基本没在原地，而是在运动，有可能是目标
                            except_boxes.append(bbox)
                    else:
                        # 第一帧的box
                        box = ball_boxes[0]

                    if len(except_boxes) > 0:
                        if max(kf.compute_iou(bbox, except_boxes)) < 0.1:
                            box = bbox
                            kf_mean, kf_covariance = kf.update(kf_mean, kf_covariance, kf.xyxy_to_xyah(box))
                    print(box)
                    x1, y1, x2, y2 = box
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                
                
                if kf_mean is None and kf_covariance is None and max(ball_confidences)>0.7 or stable_frames == 0:

                    """
                    **初始化阶段**: 当self.kf_mean和self.kf_covariance尚未初始化,
                    或者self.stable_frames为0时,使用第一个检测到的边界框来初始化卡尔曼滤波的状态(均值和协方差矩阵)
                    此时直接选择IoU最高的掩码,并初始化KF。
                    """

                    kf_mean, kf_covariance = kf.initiate(kf.xyxy_to_xyah([806.7212, 446.2137, 816.0037, 461.7293]))
                    stable_frames += 1

                elif stable_frames < stable_frames_threshold:

                    """
                    当self.stable_frames小于阈值时,卡尔曼滤波进行预测,但不进行更新,除非当前帧的IoU足够高.
                    此时,如果检测到的IoU超过阈值,则用当前边界框更新KF状态,并增加stable_frames的计数,否则重置计数。
                    """

                    kf_mean, kf_covariance = kf.predict(kf_mean, kf_covariance)
                    
                    predict_iou = kf._compute_iou(box, kf.xyah_to_xyxy(kf_mean[:4]))
                    print(kf.xyah_to_xyxy(kf_mean[:4]), box, predict_iou)
                    if predict_iou > stable_ious_threshold:
                        kf_mean, kf_covariance = kf.update(kf_mean, kf_covariance, kf.xyxy_to_xyah(box))
                        stable_frames += 1
                    else:
                        stable_frames = 0
                else:

                    """
                    **稳定后的更新阶段**:当stable_frames达到阈值后,KF进入正常预测和更新循环.
                    每次预测后, 计算当前多个候选掩码的边界框与KF预测的边界框之间的IoU, 
                    然后结合KF的IoU和模型预测的IoU,加权后选择最佳掩码,并更新KF的状态。
                    """
                                        
                    kf_mean, kf_covariance = kf.predict(kf_mean, kf_covariance)
                
                    kf_mean, kf_covariance = kf.update(kf_mean, kf_covariance, kf.xyxy_to_xyah(box))
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        
            else:
                if kf_mean is not None and kf_covariance is not None:
                    kf_mean, kf_covariance = kf.predict(kf_mean, kf_covariance)
                    box = kf.xyah_to_xyxy(kf_mean[:4])
                    
                    x1, y1, x2, y2 = box
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)

                # kf_mean, kf_covariance = kf.update(kf_mean, kf_covariance, kf.xyxy_to_xyah(box))
            # 绘制边界框
            
        # 写入输出视频
        out.write(frame)

        last_frame_boxes = ball_boxes
        frame_id += 1
        print(f"Processed frame {frame_id}. stable_frames: {stable_frames}")

    # 释放所有资源
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [40]:
model = RTDETR("rtdetr-l.pt")
process_video(video_input_path, video_output_path, model)

e:/TennisProject-main/inference/test_02.mp4

0: 640x640 15 persons, 1 bench, 1 sports ball, 4 chairs, 35.3ms
Speed: 4.0ms preprocess, 35.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
[     656.13      97.229      666.92      105.55]
Processed frame 1. stable_frames: 1

0: 640x640 14 persons, 1 bench, 4 tennis rackets, 6 chairs, 40.5ms
Speed: 3.0ms preprocess, 40.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Processed frame 2. stable_frames: 1

0: 640x640 14 persons, 1 bench, 3 tennis rackets, 4 chairs, 37.1ms
Speed: 3.6ms preprocess, 37.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Processed frame 3. stable_frames: 1

0: 640x640 15 persons, 1 bench, 2 tennis rackets, 5 chairs, 33.9ms
Speed: 4.3ms preprocess, 33.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Processed frame 4. stable_frames: 1

0: 640x640 13 persons, 1 bench, 2 tennis rackets, 6 chairs, 35.6ms
Speed: 3.1ms preprocess, 35.6ms infer

### V1

In [96]:
def process_video(video_input_path, video_output_path, model):
    kf = KalmanFilter()
    cap = cv2.VideoCapture(video_input_path)
    if not cap.isOpened():
        raise IOError("Cannot open the video file")

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 降低输出帧率以匹配处理速度
    out = cv2.VideoWriter(video_output_path, cv2.VideoWriter_fourcc(*'mp4v'), 15, (frame_width, frame_height))

    frame_id = 0
    stable_frames = 0
    stable_frames_threshold = 5  # 提高稳定阈值
    stable_ious_threshold = 0.5  # 更严格的IoU阈值
    score_weight = 0.3           # 置信度权重降低

    kf_mean, kf_covariance = None, None
    tracked_id = None            # 当前跟踪目标的唯一标识（可选）

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        best_box = None
        
        # 提取所有球体检测
        ball_boxes = []
        ball_confidences = []
        for result in results:
            if result.boxes.cls.numel() > 0:
                ball_mask = (result.boxes.cls.cpu().numpy() == 32)
                ball_boxes.extend(result.boxes.xyxy.cpu().numpy()[ball_mask])
                ball_confidences.extend(result.boxes.conf.cpu().numpy()[ball_mask])

        if len(ball_boxes) > 0:
            # --- 数据关联逻辑 ---
            if kf_mean is not None:
                # 仅预测一次
                kf_mean_pred, kf_cov_pred = kf.predict(kf_mean, kf_covariance)
                scores = []
                
                for bbox, conf in zip(ball_boxes, ball_confidences):
                    # 计算马氏距离
                    mahalanobis_dist = compute_mahalanobis(kf_mean_pred, kf_cov_pred, bbox)
                    
                    # 计算IoU
                    kf_pred_bbox = kf.xyah_to_xyxy(kf_mean_pred[:4])
                    iou = kf._compute_iou(bbox, kf_pred_bbox)
                    
                    # 归一化评分
                    maha_similarity = max(0, 1 - (mahalanobis_dist / 5.0))  # 马氏距离阈值设为5
                    iou_similarity = iou
                    
                    # 加权综合得分（置信度权重降低）
                    score = (score_weight * conf + 
                           0.4 * maha_similarity + 
                           0.3 * iou_similarity)
                    scores.append(score)

                best_idx = np.argmax(scores) if scores else -1
                if best_idx != -1 and scores[best_idx] > 0.4:  # 总体相似度阈值
                    best_box = ball_boxes[best_idx]
                    
            # --- 初始化或更新逻辑 ---
            if best_box is None:
                # 选择最高置信度的检测初始化
                best_idx = np.argmax(ball_confidences)
                best_box = ball_boxes[best_idx]

            # 首次初始化
            if kf_mean is None:
                kf_mean, kf_covariance = kf.initiate(kf.xyxy_to_xyah(best_box))
                stable_frames = 1
                print(f"KF Initialized with box: {best_box}")
            else:
                # 更新稳定帧计数
                current_iou = kf._compute_iou(best_box, kf.xyah_to_xyxy(kf_mean[:4]))
                if current_iou >= stable_ious_threshold:
                    stable_frames = min(stable_frames + 1, stable_frames_threshold)
                else:
                    stable_frames = max(stable_frames - 2, 0)  # 快速惩罚不稳定帧

                # 更新KF状态
                kf_mean, kf_covariance = kf.update(kf_mean_pred, kf_cov_pred, kf.xyxy_to_xyah(best_box))

            # 绘制最佳框
            x1, y1, x2, y2 = map(int, best_box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # 绘制KF预测框
            # 假设 kf_mean[:4] 是一个包含四个元素的列表
            kf_bbox = np.array(kf.xyah_to_xyxy(kf_mean[:4])).astype(int)
            cv2.rectangle(frame, tuple(kf_bbox[:2]), tuple(kf_bbox[2:]), (0, 0, 255), 2)
            
        else:
            # 无检测时仅预测
            if kf_mean is not None:
                kf_mean, kf_covariance = kf.predict(kf_mean, kf_covariance)
                stable_frames = max(stable_frames - 1, 0)
                kf_bbox = np.array(kf.xyah_to_xyxy(kf_mean[:4])).astype(int)
                cv2.rectangle(frame, tuple(kf_bbox[:2]), tuple(kf_bbox[2:]), (255, 255, 255), 2)

        # 显示稳定帧计数
        cv2.putText(frame, f"Stable: {stable_frames}/{stable_frames_threshold}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        out.write(frame)
        frame_id += 1
        print(f"Frame {frame_id}: Stable={stable_frames}")

    cap.release()
    out.release()

### RAW VIDEO

In [23]:
def process_video(video_input_path, video_output_path, model):
    kf = KalmanFilter()
    
    print(video_input_path)
    # 打开视频文件
    cap = cv2.VideoCapture(video_input_path)
    if not cap.isOpened():
        raise IOError("Cannot open the video file")

    # 获取视频参数以复制编解码器和帧率
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # 获取视频帧率
    frame_id = 0

    # 初始化视频写入器
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_output_path, fourcc, 1, (frame_width, frame_height))

    while True:
        # 读取视频帧
        ret, frame = cap.read()
        if not ret:
            break  # 视频结束

        results = model(frame)
        for result in results:
            # print(result.boxes)

            if 32 in result.boxes.cls:
                indexs = torch.where(result.boxes.cls==32)

                for index in indexs:
                    print(result.boxes.xyxy[index], result.boxes.conf[index], index)
                    boxes =  result.boxes.xyxy[index]
                    print(result.boxes.id)
                    confidences = result.boxes.conf[index]
                    for _, (box, confidence) in enumerate(zip(boxes, confidences)):
                        box = box.cpu().numpy()

                        x1, y1, x2, y2 = box
                        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        text = f"{confidence:.2f}"  # 格式化置信度到小数点后两位
                        cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        

            
        # 写入输出视频
        out.write(frame)
        frame_id += 1
        print(f"Frame {frame_id}")
    # 释放所有资源
    cap.release()
    out.release()
    cv2.destroyAllWindows()

video_input_path = 'e:/TennisProject-main/inference/test.mp4'
video_output_path = 'e:/tennis_v2/inference/raw_' + video_input_path.split('/')[-1]
video_output_path

model = RTDETR("rtdetr-l.pt")
process_video(video_input_path, video_output_path, model)


e:/TennisProject-main/inference/test.mp4

0: 640x640 16 persons, 2 tennis rackets, 5 chairs, 1 tv, 3 clocks, 113.7ms
Speed: 5.0ms preprocess, 113.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Frame 1

0: 640x640 16 persons, 1 sports ball, 3 tennis rackets, 10 chairs, 1 tv, 3 clocks, 49.0ms
Speed: 3.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
tensor([[642.2836, 556.8745, 648.1731, 562.3210]], device='cuda:0') tensor([0.2697], device='cuda:0') tensor([18], device='cuda:0')
None
Frame 2

0: 640x640 17 persons, 1 sports ball, 2 tennis rackets, 11 chairs, 1 tv, 3 clocks, 80.5ms
Speed: 3.0ms preprocess, 80.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
tensor([[642.3456, 556.8234, 648.2603, 562.4624]], device='cuda:0') tensor([0.2639], device='cuda:0') tensor([16], device='cuda:0')
None
Frame 3

0: 640x640 16 persons, 1 sports ball, 2 tennis rackets, 10 chairs, 3 clocks, 63.0ms
Speed: 3.0ms preprocess, 6